In [1]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import sys
from scipy.sparse import linalg
from pathlib import Path
import itertools
if torch.cuda.is_available():  
    device = "cuda" 
else:  
    device = "cpu" 


torch.set_default_dtype(torch.float64)
pi = torch.tensor(np.pi,dtype=torch.float64)
zero = torch.tensor([0.]).to(device)

class model(nn.Module):
    """ ReLU k shallow neural network
    Parameters: 
    input size: input dimension
    hidden_size1 : number of hidden layers 
    num_classes: output classes 
    k: degree of relu functions
    """
    def __init__(self, input_size, hidden_size1, num_classes,k = 1):
        super().__init__()
        self.fc1 = nn.Linear(input_size, hidden_size1)
        self.fc2 = nn.Linear(hidden_size1, num_classes,bias = False)
        self.k = k 
    def forward(self, x):
        u1 = self.fc2(F.relu(self.fc1(x))**self.k)
        return u1
    def evaluate_derivative(self, x, i):
        if self.k == 1:
            u1 = self.fc2(torch.heaviside(self.fc1(x),zero) * self.fc1.weight.t()[i-1:i,:] )
        else:
            u1 = self.fc2(self.k*F.relu(self.fc1(x))**(self.k-1) *self.fc1.weight.t()[i-1:i,:] )  
        return u1

def plot_2D(f): 
    
    Nx = 400
    Ny = 400 
    xs = np.linspace(0, 1, Nx)
    ys = np.linspace(0, 1, Ny)
    x, y = np.meshgrid(xs, ys, indexing='xy')
    xy_comb = np.stack((x.flatten(),y.flatten())).T
    xy_comb = torch.tensor(xy_comb)
    z = f(xy_comb).reshape(Nx,Ny)
    z = z.detach().numpy()
    plt.figure(dpi=200)
    ax = plt.axes(projection='3d')
    ax.plot_surface(x , y , z )

    plt.show()

def plot_subdomains(my_model):
    x_coord =torch.linspace(0,1,200)
    wi = my_model.fc1.weight.data
    bi = my_model.fc1.bias.data 
    for i, bias in enumerate(bi):  
        if wi[i,1] !=0: 
            plt.plot(x_coord, - wi[i,0]/wi[i,1]*x_coord - bias/wi[i,1])
        else: 
            plt.plot(x_coord,  - bias/wi[i,0]*torch.ones(x_coord.size()))

    plt.xlim([0,1])
    plt.ylim([0,1])
    plt.legend()
    plt.show()
    return 0   

def adjust_neuron_position(my_model, dims = 3):

    def create_mesh_grid(dims, pts):
        mesh = torch.tensor(list(itertools.product(pts,repeat=dims)))
        vertices = mesh.reshape(len(pts) ** dims, -1) 
        return vertices
    counter = 0 
    # positions = torch.tensor([[0.,0.],[0.,1.],[1.,1.],[1.,0.]])
    pts = torch.tensor([0.,1.])
    positions = create_mesh_grid(dims,pts) 
    neuron_num = my_model.fc1.bias.size(0)
    for i in range(neuron_num): 
        w = my_model.fc1.weight.data[i:i+1,:]
        b = my_model.fc1.bias.data[i]
    #     print(w,b)
        values = torch.matmul(positions,w.T) # + b
        left_end = - torch.max(values)
        right_end = - torch.min(values)
        offset = (right_end - left_end)/50
        if b <= left_end + offset/2 : 
            b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
            my_model.fc1.bias.data[i] = b 
        if b >= right_end - offset/2 :
            if counter < (dims+1):
#                 print("here")
                counter += 1
            else: # (d + 1) or more 
                b = torch.rand(1)*(right_end - left_end - offset) + left_end + offset/2 
                my_model.fc1.bias.data[i] = b 
    return my_model



In [7]:
def MonteCarlo_Sobol_dDim_weights_points(M ,d = 4):
    Sob_integral = torch.quasirandom.SobolEngine(dimension =d, scramble= False, seed=None) 
    integration_points = Sob_integral.draw(M).double() 
    integration_points = integration_points.to(device)
    weights = torch.ones(M,1).to(device)/M 
    return weights, integration_points 

def generate_relu_dict4D(N_list):
    
    N = np.prod(N_list) 

    grid_indices = [np.linspace(0,1,N_item,endpoint=False) for N_item in N_list]
    grid = np.meshgrid(*grid_indices,indexing='ij')
    grid_coordinates = np.column_stack([axis.ravel() for axis in grid]) 
    samples = torch.tensor(grid_coordinates) 

    T =torch.tensor([[pi,0,0,0],[0,pi,0,0],[0,0,2*pi,0],[0,0,0,2*2]]) # 2 * sqrt(d)
    shift = torch.tensor([0,0,0,-2])
    samples = samples@T + shift 

    f1 = torch.zeros(N,1) 
    f2 = torch.zeros(N,1)
    f3 = torch.zeros(N,1)
    f4 = torch.zeros(N,1)
    f5 = torch.zeros(N,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.cos(samples[:,2])
    f4[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.sin(samples[:,2])  
    f5[:,0] = samples[:,3]

    Wb_tensor = torch.cat([f1,f2,f3,f4,f5],1) # N x 4 
    return Wb_tensor

def PiecewiseGQ3D_weights_points(Nx, order): 
    """ A slight modification of PiecewiseGQ2D function that only needs the weights and integration points.
    Parameters
    ----------

    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature

    Returns
    -------
    long_weights: torch.tensor
    integration_points: torch.tensor
    """

    """
    Parameters
    ----------
    target : 
        Target function 
    Nx: int 
        number of intervals along the dimension. No Ny, assume Nx = Ny
    order: int 
        order of the Gauss Quadrature
    """

    # print("order: ",order )
    x, w = np.polynomial.legendre.leggauss(order)
    gauss_pts = np.array(np.meshgrid(x,x,x,indexing='ij')).reshape(3,-1).T
    weight_list = np.array(np.meshgrid(w,w,w,indexing='ij'))
    weights =   (weight_list[0]*weight_list[1]*weight_list[2]).ravel() 

    gauss_pts =torch.tensor(gauss_pts)
    weights = torch.tensor(weights)

    h = 1/Nx # 100 intervals 
    long_weights =  torch.tile(weights,(Nx**3,1))
    long_weights = long_weights.reshape(-1,1)
    long_weights = long_weights * h**3 /8 

    integration_points = torch.tile(gauss_pts,(Nx**3,1))
    # print("shape of integration_points", integration_points.size())
    scale_factor = h/2 
    integration_points = scale_factor * integration_points

    index = np.arange(1,Nx+1)-0.5
    ordered_pairs = np.array(np.meshgrid(index,index,index,indexing='ij'))
    ordered_pairs = ordered_pairs.reshape(3,-1).T

    # print(ordered_pairs)
    # print()
    ordered_pairs = torch.tensor(ordered_pairs)
    # print(ordered_pairs.size())
    ordered_pairs = torch.tile(ordered_pairs, (1,order**3)) # number of GQ points
    # print(ordered_pairs)

    ordered_pairs =  ordered_pairs.reshape(-1,3)
    # print(ordered_pairs)
    translation = ordered_pairs*h 
    # print(translation)

    integration_points = integration_points + translation 

    return long_weights.to(device), integration_points.to(device)


def generate_relu_dict4D_QMC(s,N0):
    # Sob = torch.quasirandom.SobolEngine(dimension =4, scramble= True, seed=None) 
    # samples = Sob.draw(N0).double() 

    # for i in range(s-1):
    #     samples = torch.cat([samples,Sob.draw(N0).double()],0)

    # Monte Carlo 
    samples = torch.rand(s*N0,4) 

    T =torch.tensor([[pi,0,0,0],[0,pi,0,0],[0,0,2*pi,0],[0,0,0,2*2]])
    shift = torch.tensor([0,0,0,-2])
    samples = samples@T + shift 

    f1 = torch.zeros(s*N0,1) 
    f2 = torch.zeros(s*N0,1)
    f3 = torch.zeros(s*N0,1)
    f4 = torch.zeros(s*N0,1)
    f5 = torch.zeros(s*N0,1)

    f1[:,0] = torch.cos(samples[:,0]) 
    f2[:,0] = torch.sin(samples[:,0]) * torch.cos(samples[:,1])
    f3[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.cos(samples[:,2])
    f4[:,0] = torch.sin(samples[:,0]) * torch.sin(samples[:,1]) * torch.sin(samples[:,2])  
    f5[:,0] = samples[:,3]

    Wb_tensor = torch.cat([f1,f2,f3,f4,f5],1) # N x 4 
    return Wb_tensor

def minimize_linear_layer_H1_explicit_assemble_efficient(model,alpha, target, g_N, weights, integration_points, w_bd, pts_bd, activation = 'relu',solver="direct" ):
    """ -div alpha grad u(x) + u = f 
    Parameters
    ----------
    model: 
        nn model
    alpha:
        alpha function
    target:
        rhs function f 
    pts_bd:
        integration points on the boundary, embdedded in the domain 
    """ 
    zero = torch.tensor([0.]).to(device)
    start_time = time.time() 
    w = model.fc1.weight.data 
    b = model.fc1.bias.data 
    neuron_num = b.size(0) 
    dim = integration_points.size(1) 
    
    coef_alpha = alpha(integration_points) # alpha  
    if activation == 'relu':
        basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k)
        weighted_basis_value_col = basis_value_col * weights 
        jac = weighted_basis_value_col.t() @ basis_value_col  # mass matrix 
        rhs = weighted_basis_value_col.t() @ (target(integration_points)) 

        # Todo1: assemble the boundary condition term <g,v>_{\Gamma_N} 
        size_pts_bd = int(pts_bd.size(0)/(2*dim))
        if g_N != None:
            bcs_N = g_N(dim)
            for ii, g_ii in bcs_N:
                # pts_bd_ii = pts_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
                weighted_g_N = -g_ii(pts_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:])* w_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
                basis_value_bd_col = F.relu(pts_bd[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:] @ w.t()+ b)**(model.k)
                rhs += basis_value_bd_col.t() @ weighted_g_N

                weighted_g_N = g_ii(pts_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:])* w_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:]
                basis_value_bd_col = F.relu(pts_bd[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:] @ w.t()+ b)**(model.k)
                rhs += basis_value_bd_col.t() @ weighted_g_N
        
        if model.k == 1:  
            for d in range(dim):
                basis_value_dxi_col = torch.heaviside(integration_points @ w.t()+ b, zero) * w.t()[d:d+1,:]
                weighted_basis_value_dx_col = basis_value_dxi_col * weights * coef_alpha 
                jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
#             basis_value_dx_all_col = torch.stack([torch.heaviside(integration_points @ w.t()+ b, zero) * w.t()[d:d+1,:] for d in range(dim)])
            
        else: 
            for d in range(dim):
                basis_value_dxi_col = model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[d:d+1,:]
                weighted_basis_value_dx_col = basis_value_dxi_col * weights * coef_alpha  
                jac += weighted_basis_value_dx_col.t() @ basis_value_dxi_col 
#             basis_value_dx_all_col = torch.stack([model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[d:d+1,:] for d in range(dim)]) 
            # basis_value_dx_col = model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[0:1,:]
            # basis_value_dy_col = model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[1:2,:] 

    print("assembling the mass matrix time taken: ", time.time()-start_time) 


    start_time = time.time()    
    if solver == "cg": 
        sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
        sol = torch.tensor(sol).view(1,-1)
    elif solver == "direct": 
#         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
        sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
    elif solver == "ls":
        sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
        # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
    print("solving Ax = b time taken: ", time.time()-start_time)
    return sol 

# def minimize_linear_layer_H1_explicit_assemble_efficient(model,alpha, target,weights, integration_points,activation = 'relu',solver="direct" ):
#     """ -div alpha grad u(x) + u = f 
#     Parameters
#     ----------
#     model: 
#         nn model
#     alpha:
#         alpha function
#     target:
#         rhs function f 
#     """ 
#     zero = torch.tensor([0.]).to(device)
#     start_time = time.time() 
#     w = model.fc1.weight.data 
#     b = model.fc1.bias.data 
#     neuron_num = b.size(0) 
#     dim = integration_points.size(1) 

#     if activation == 'relu':
#         basis_value_col = F.relu(integration_points @ w.t()+ b)**(model.k) 
#         if model.k == 1:  
#             basis_value_dx_all_col = torch.stack([torch.heaviside(integration_points @ w.t()+ b, zero) * w.t()[d:d+1,:] for d in range(dim)])
            
#         else: 
#             basis_value_dx_all_col = torch.stack([model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[d:d+1,:] for d in range(dim)]) 
#             # basis_value_dx_col = model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[0:1,:]
#             # basis_value_dy_col = model.k * F.relu(integration_points @ w.t()+ b)**(model.k-1) * w.t()[1:2,:] 

#     weighted_basis_value_col = basis_value_col * weights 
#     jac = weighted_basis_value_col.t() @ basis_value_col  # mass matrix 
#     rhs = weighted_basis_value_col.t() @ (target(integration_points)) 
#     print("assembling the mass matrix time taken: ", time.time()-start_time) 

#     start_time = time.time() 
#     coef_alpha = alpha(integration_points) # alpha 
#     for basis_value_col_dx in basis_value_dx_all_col: 
#         weighted_basis_value_dx_col = basis_value_col_dx * weights * coef_alpha # alpha 
#         jac += weighted_basis_value_dx_col.t() @ basis_value_col_dx
#     # weighted_basis_value_dx_col = basis_value_dx_col * weights
#     # weighted_basis_value_dy_col = basis_value_dy_col * weights
#     # jac2 = weighted_basis_value_dx_col.t() @ basis_value_dx_col + weighted_basis_value_dy_col.t() @ basis_value_dy_col 
#     # print("assembling the stiffness matrix time taken: ", time.time()-start_time)   
#     # jac = jac1 + jac2    
    
#     start_time = time.time()    
#     if solver == "cg": 
#         sol, exit_code = linalg.cg(np.array(jac.detach().cpu()),np.array(rhs.detach().cpu()),tol=1e-12)
#         sol = torch.tensor(sol).view(1,-1)
#     elif solver == "direct": 
# #         sol = np.linalg.inv( np.array(jac.detach().cpu()) )@np.array(rhs.detach().cpu())
#         sol = (torch.linalg.solve( jac.detach(), rhs.detach())).view(1,-1)
#     elif solver == "ls":
#         sol = (torch.linalg.lstsq(jac.detach().cpu(),rhs.detach().cpu(),driver='gelsd').solution).view(1,-1)
#         # sol = (torch.linalg.lstsq(jac.detach(),rhs.detach()).solution).view(1,-1) # gpu/cpu, driver = 'gels', cannot solve singular
#     print("solving Ax = b time taken: ", time.time()-start_time)
#     return sol 


def OGANeumannReLU4D(my_model,alpha, target,g_N, u_exact, u_exact_grad, N_list,num_epochs,plot_freq, M, k =1, rand_deter = 'deter', linear_solver = "direct"): 
    """ Orthogonal greedy algorithm using 1D ReLU dictionary over [-pi,pi]
    Parameters
    ----------
    my_model: 
        nn model 
    target: 
        target function
    num_epochs: int 
        number of training epochs 
    integration_intervals: int 
        number of subintervals for piecewise numerical quadrature 

    Returns
    -------
    err: tensor 
        rank 1 torch tensor to record the L2 error history  
    model: 
        trained nn model 
    """
    #Todo Done
    dim = 4 
    gw_expand, integration_points = MonteCarlo_Sobol_dDim_weights_points(M, d=dim)
    gw_expand = gw_expand.to(device)
    integration_points = integration_points.to(device)

    # define integration on the boundary 
    gw_expand_bd, integration_points_bd = PiecewiseGQ3D_weights_points(25, order = 2) 
    size_pts_bd = integration_points_bd.size(0) 
    gw_expand_bd_faces = torch.tile(gw_expand_bd,(2*dim,1))

    integration_points_bd_faces = torch.zeros(2*dim*integration_points_bd.size(0),dim).to(device)
    integration_points_bd_faces[0:size_pts_bd,0:1] = 0 
    integration_points_bd_faces[0:size_pts_bd,1:] = integration_points_bd[:]

    integration_points_bd_faces[size_pts_bd:size_pts_bd*2,0:1] = 1
    integration_points_bd_faces[size_pts_bd:size_pts_bd*2,1:] = integration_points_bd[:,:]

    integration_points_bd_faces[size_pts_bd*2:size_pts_bd*3,1:2] = 0 
    integration_points_bd_faces[size_pts_bd*2:size_pts_bd*3, 0:1] = integration_points_bd[:,0:1]
    integration_points_bd_faces[size_pts_bd*2:size_pts_bd*3,2:] =  integration_points_bd[:,1:]

    integration_points_bd_faces[size_pts_bd*3:size_pts_bd*4,1:2] = 1
    integration_points_bd_faces[size_pts_bd*3:size_pts_bd*4, 0:1] = integration_points_bd[:,0:1]
    integration_points_bd_faces[size_pts_bd*3:size_pts_bd*4,2:] =  integration_points_bd[:,1:]

    integration_points_bd_faces[size_pts_bd*4:size_pts_bd*5,2:3] = 0
    integration_points_bd_faces[size_pts_bd*4:size_pts_bd*5,0:2] = integration_points_bd[:,0:2]
    integration_points_bd_faces[size_pts_bd*4:size_pts_bd*5,3:] = integration_points_bd[:,2:]

    integration_points_bd_faces[size_pts_bd*5:size_pts_bd*6,2:3] = 1
    integration_points_bd_faces[size_pts_bd*5:size_pts_bd*6,0:2] = integration_points_bd[:,0:2]
    integration_points_bd_faces[size_pts_bd*5:size_pts_bd*6,3:] = integration_points_bd[:,2:]

    integration_points_bd_faces[size_pts_bd*6:size_pts_bd*7,3:4] = 0
    integration_points_bd_faces[size_pts_bd*6:size_pts_bd*7,0:3] = integration_points_bd[:,:]

    integration_points_bd_faces[size_pts_bd*7:size_pts_bd*8,3:4] = 1 
    integration_points_bd_faces[size_pts_bd*7:size_pts_bd*8,0:3] = integration_points_bd[:,:]

    err = torch.zeros(num_epochs+1)
    err_h10 = torch.zeros(num_epochs+1) 
    if my_model == None: 
        func_values = - target(integration_points)
        num_neuron = 0

        list_b = []
        list_w = []
    else: 
        func_values = - (target(integration_points) - my_model(integration_points).detach())
        bias = my_model.fc1.bias.detach().data
        weights = my_model.fc1.weight.detach().data
        num_neuron = int(bias.size(0))

        list_b = list(bias)
        list_w = list(weights)
    
    # initial error Todo Done
    func_values_sqrd = func_values*func_values
    # print(func_values_sqrd.size())
    # print(gw_expand.size()) 
    err[0]= torch.sum(func_values_sqrd*gw_expand)**0.5
    ## h1 seminorm 
    if u_exact_grad != None:
        u_grad = u_exact_grad() 
        for grad_i in u_grad: 
            err_h10[0] += torch.sum((grad_i(integration_points))**2 * gw_expand)**0.5
    
    start_time = time.time()
    solver = linear_solver

    N0 = np.prod(N_list)
    if rand_deter == 'deter':
        relu_dict_parameters = generate_relu_dict4D(N_list).to(device)
    print("using linear solver: ",solver)

    for i in range(num_epochs): 
        print("epoch: ",i+1, end = '\t')
        if rand_deter == 'rand':
            relu_dict_parameters = generate_relu_dict4D_QMC(1,N0).to(device) 
        if num_neuron == 0: 
            func_values = - target(integration_points)
        else: 
            func_values = - target(integration_points) + my_model(integration_points).detach()

        weight_func_values = func_values*gw_expand  
        basis_values = (F.relu( torch.matmul(integration_points,relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T # uses broadcasting, # dimension 4 
        output = torch.matmul(basis_values,weight_func_values) #
        
        # grad u part
        alpha_coef = alpha(integration_points) # alpha 
        if my_model!= None:
            if k == 1:  
                derivative_part = torch.heaviside(integration_points @ (relu_dict_parameters[:,0:4].T) - relu_dict_parameters[:,4], zero) # dimension 4 
                derivative_part *= alpha_coef # alpha 
                for dx_i in range(dim): 
                    weight_dbasis_values_dxi =  (derivative_part * relu_dict_parameters.t()[dx_i:dx_i+1,:]) *gw_expand   
                    dmy_model_dxi = my_model.evaluate_derivative(integration_points,dx_i+1).detach()
                    output += torch.matmul(weight_dbasis_values_dxi.t(), dmy_model_dxi) 

            else:  
                derivative_part = k * F.relu(integration_points @ (relu_dict_parameters[:,0:4].T) - relu_dict_parameters[:,4])**(k-1) # dimension 4 
                derivative_part *= alpha_coef # alpha
                for dx_i in range(dim): 
                    weight_dbasis_values_dxi =  (derivative_part * relu_dict_parameters.t()[dx_i:dx_i+1,:]) * gw_expand    
                    dmy_model_dxi = my_model.evaluate_derivative(integration_points,dx_i+1).detach()
                    output += torch.matmul(weight_dbasis_values_dxi.t(), dmy_model_dxi) 
        
        #Todo2 boundary condition term -<g,v>_{\Gamma_N}  
        if g_N != None:
            bcs_N = g_N(dim) 
            for ii, g_ii in bcs_N: 
                
                weighted_g_N = -g_ii(integration_points_bd_faces[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:])* gw_expand_bd_faces[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:]
                basis_values_bd_faces = (F.relu( torch.matmul(integration_points_bd_faces[2*ii*size_pts_bd:(2*ii+1)*size_pts_bd,:],relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T
                output -= torch.matmul(basis_values_bd_faces,weighted_g_N)
                
                weighted_g_N = g_ii(integration_points_bd_faces[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:])* gw_expand_bd_faces[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:]
                basis_values_bd_faces = (F.relu( torch.matmul(integration_points_bd_faces[(2*ii+1)*size_pts_bd:(2*ii+2)*size_pts_bd,:],relu_dict_parameters[:,0:4].T ) - relu_dict_parameters[:,4])**k).T
                output -= torch.matmul(basis_values_bd_faces,weighted_g_N)

        
        # output = torch.abs(torch.matmul(basis_values,weight_func_values)) # 
        output = torch.abs(output)
        neuron_index = torch.argmax(output.flatten())
        
        # print(neuron_index)
        list_w.append(relu_dict_parameters[neuron_index,0:4]) # dimension 4 
        list_b.append(-relu_dict_parameters[neuron_index,4])
        num_neuron += 1
        my_model = model(4,num_neuron,1,k).to(device)
        w_tensor = torch.stack(list_w, 0 ) 
        b_tensor = torch.tensor(list_b)
        my_model.fc1.weight.data[:,:] = w_tensor[:,:]
        my_model.fc1.bias.data[:] = b_tensor[:]

        #Todo Done 
#         alpha = None 
        sol = minimize_linear_layer_H1_explicit_assemble_efficient(my_model,alpha, target, g_N, gw_expand, integration_points,gw_expand_bd_faces, integration_points_bd_faces,activation = 'relu',solver = solver)

        my_model.fc2.weight.data[0,:] = sol[:]

        model_values = my_model(integration_points).detach()
        # L2 error ||u - u_n||
        diff_values_sqrd = (u_exact(integration_points) - model_values)**2 
        err[i+1]= torch.sum(diff_values_sqrd*gw_expand)**0.5

        # H10 error || grad(u) - grad(u_n) ||
        if u_exact_grad != None:
            for ii, grad_i in enumerate(u_grad):  
                my_model_dxi = my_model.evaluate_derivative(integration_points,ii+1).detach()  
                err_h10[i+1] += torch.sum((grad_i(integration_points) - my_model_dxi)**2 * gw_expand)**0.5

    print("time taken: ",time.time() - start_time)
    return err, err_h10, my_model




In the following tests, we compare using deterministic dictionaries with using random dictionary for the following three target functions. 

- $\sin(\pi x_1) \sin(\pi x_2)$ 
- $\sin(4\pi x_1) \sin(8\pi x_2)$ 
- Gabor function 

In [8]:
# gw_expand, integration_points =MonteCarlo_Sobol_dDim_weights_points(600000, d=4)
# my_model = model(4,320,1,1).to(device)
# my_model = adjust_neuron_position(my_model.cpu(),dims = 4).to(device) 
# def u_exact(x):
#     return torch.cos(pi*x[:,0:1])*torch.cos(pi*x[:,1:2] )*torch.cos(pi*x[:,2:3])*torch.cos(pi*x[:,3:4])
# def target(x):
#     z = (4 * pi**2 + 1)*torch.cos(pi*x[:,0:1])*torch.cos(pi*x[:,1:2] )*torch.cos(pi*x[:,2:3])*torch.cos(pi*x[:,3:4]) 
#     return z  
# solver = 'direct'
# alpha = None 
# sol = minimize_linear_layer_H1_explicit_assemble_efficient(my_model,alpha, target,gw_expand, integration_points,activation = 'relu',solver = solver)
# my_model.fc2.weight.data[0,:] = sol[:] 

# diff_values = (u_exact(integration_points) - my_model(integration_points).detach())**2 
# integral = torch.sum(diff_values*gw_expand)**0.5 
# print("L2 error: ",integral) 


In [17]:
def u_exact(x):
    d = 4 
    cn =   7.03/d 
    return torch.exp(-torch.sum( cn**2 * (x - 0.5)**2,dim = 1, keepdim = True))  

def u_exact_grad():
    d = 4 
    cn = 7.03/d
    def make_grad_i(i):
        def grad_i(x):
            return torch.exp(-torch.sum(cn**2 * (x - 0.5)**2, dim=1, keepdim=True)) * (-2 * cn**2 * (x[:, i:i+1] - 0.5))
        return grad_i 
    
    u_grad=[] 
    for i in range(d):
        u_grad.append(make_grad_i(i))
    return u_grad


def alpha(x): 
    return torch.ones(x.size(0),1).to(device)
    # return 0.5 * torch.sin(6 * pi*x[:,0:1]) + 1. 

# def target(x):
# #     z = (  4 * (pi)**2 + 1)*torch.cos( pi*x[:,0:1])*torch.cos( pi*x[:,1:2] ) * torch.cos(pi*x[:,2:3]) * torch.cos( pi*x[:,3:4]) 
# #     return z 
#     z_c = torch.cos( pi*x[:,0:1])*torch.cos( pi*x[:,1:2] ) * torch.cos(pi*x[:,2:3]) * torch.cos( pi*x[:,3:4]) 
#     z1 = 3 * pi**2 * torch.sin(pi * x[:,0:1]) * torch.cos( 6*pi*x[:,1:2] ) * torch.cos(pi*x[:,2:3]) * torch.cos( pi*x[:,3:4]) 
#     z2 = 0.5 * pi**2 * torch.sin(6*pi * x[:,0:1])* z_c 
#     z = z1 + z2 + 3/2*pi**2 * torch.sin(6 * pi * x[:,0:1]) * z_c 
#     z += ( 4 * (pi)**2 + 1)*z_c 
#     return z 

def target(x):
    d = 4 
    cn =   7.03/d 
    z = torch.exp(-torch.sum( cn**2 * (x - 0.5)**2,dim = 1, keepdim = True)) 
    return z* ( -torch.sum(  (2 *cn**2 * (x - 0.5))**2 - 2*cn**2 ,dim = 1, keepdim = True) +1)

def g_N(dim):
    def make_g(i):
        def g_i(x):
            d = 4 
            cn = 7.03 / d
            return torch.exp(-torch.sum(cn**2 * (x - 0.5)**2, dim=1, keepdim=True)) * (-2 * cn**2 * (x[:, i:i+1] - 0.5))
        return g_i

    bcs_N = []
    for i in range(dim):
        bcs_N.append((i, make_g(i)))
    
    return bcs_N


function_name = "cospix" 
filename_write = "data-neumann/4DOGA-{}-order.txt".format(function_name)
f_write = open(filename_write, "a")
f_write.write("\n")
f_write.close() 
save = False 
for N_list in [[2**2,2**2,2**3,2**3]]: # ,[2**6,2**6],[2**7,2**7] 
    # save = True 
    f_write = open(filename_write, "a")
    my_model = None 
    # Nx = 50   
    # order = 3   
    M = 500000  
    exponent = 9
    num_epochs = 2**exponent  
    plot_freq = num_epochs 
    N = np.prod(N_list)
    err_QMC2, err_h10, my_model = OGANeumannReLU4D(my_model,alpha, target,g_N, u_exact,u_exact_grad, N_list,num_epochs,plot_freq, M, k = 2, rand_deter= 'rand', linear_solver = "direct")
    
    if save: 
        folder = 'data-neumann/'
        filename = folder + 'err_OGA_4D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(err_QMC2,filename) 
        folder = 'data-neumann/'
        filename = folder + 'model_OGA_4D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(my_model,filename)

    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_QMC2[i] for i in neuron_nums ]
    err_list2 = [err_h10[i] for i in neuron_nums ] 
    f_write.write('deterministic dictionary size: {}\n'.format(N))
    f_write.write("neuron num \t\t error \t\t order \t\t h10 error \\ order \n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t")
            
            # print("*")
            print("{} \t\t {} \t\t * \t\t {} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ) )   
            f_write.write("{} \t\t {} \t\t * \t\t {} \t\t * \n".format(neuron_nums[i],item, err_list2[i] ))
        else: 
            # print(neuron_nums[i], end = "\t\t")
            # print(item, end = "\t\t") 
            # print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            print("{} \t\t {} \t\t {} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ) )
            f_write.write("{} \t\t {} \t\t {} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2),err_list2[i] , np.log(err_list2[i-1]/err_list2[i])/np.log(2) ))
    f_write.write("\n")
    f_write.close()


using linear solver:  direct
epoch:  1	assembling the mass matrix time taken:  0.0034203529357910156
solving Ax = b time taken:  0.0005359649658203125
epoch:  2	assembling the mass matrix time taken:  0.0036268234252929688
solving Ax = b time taken:  0.0037882328033447266
epoch:  3	assembling the mass matrix time taken:  0.01637721061706543
solving Ax = b time taken:  0.0036802291870117188
epoch:  4	assembling the mass matrix time taken:  0.016672372817993164
solving Ax = b time taken:  0.004155635833740234
epoch:  5	assembling the mass matrix time taken:  0.0048792362213134766
solving Ax = b time taken:  0.004472970962524414
epoch:  6	assembling the mass matrix time taken:  0.016253948211669922
solving Ax = b time taken:  0.005098819732666016
epoch:  7	assembling the mass matrix time taken:  0.01663374900817871
solving Ax = b time taken:  0.00569462776184082
epoch:  8	assembling the mass matrix time taken:  0.016825437545776367
solving Ax = b time taken:  0.006047725677490234
epoch:  

epoch:  69	assembling the mass matrix time taken:  0.017197847366333008
solving Ax = b time taken:  0.045491695404052734
epoch:  70	assembling the mass matrix time taken:  0.003973484039306641
solving Ax = b time taken:  0.045923709869384766
epoch:  71	assembling the mass matrix time taken:  0.004012107849121094
solving Ax = b time taken:  0.04674124717712402
epoch:  72	assembling the mass matrix time taken:  0.004068851470947266
solving Ax = b time taken:  0.0461728572845459
epoch:  73	assembling the mass matrix time taken:  0.004059553146362305
solving Ax = b time taken:  0.04766654968261719
epoch:  74	assembling the mass matrix time taken:  0.004382133483886719
solving Ax = b time taken:  0.04758954048156738
epoch:  75	assembling the mass matrix time taken:  0.003870725631713867
solving Ax = b time taken:  0.04887032508850098
epoch:  76	assembling the mass matrix time taken:  0.003991365432739258
solving Ax = b time taken:  0.04823946952819824
epoch:  77	assembling the mass matrix t

epoch:  137	assembling the mass matrix time taken:  0.004236459732055664
solving Ax = b time taken:  0.09601402282714844
epoch:  138	assembling the mass matrix time taken:  0.004198551177978516
solving Ax = b time taken:  0.09595489501953125
epoch:  139	assembling the mass matrix time taken:  0.004119873046875
solving Ax = b time taken:  0.09740328788757324
epoch:  140	assembling the mass matrix time taken:  0.00420069694519043
solving Ax = b time taken:  0.09571146965026855
epoch:  141	assembling the mass matrix time taken:  0.0042095184326171875
solving Ax = b time taken:  0.09818696975708008
epoch:  142	assembling the mass matrix time taken:  0.004068613052368164
solving Ax = b time taken:  0.09814190864562988
epoch:  143	assembling the mass matrix time taken:  0.0041959285736083984
solving Ax = b time taken:  0.09940052032470703
epoch:  144	assembling the mass matrix time taken:  0.004589080810546875
solving Ax = b time taken:  0.09694528579711914
epoch:  145	assembling the mass ma

epoch:  205	assembling the mass matrix time taken:  0.0047414302825927734
solving Ax = b time taken:  0.146925687789917
epoch:  206	assembling the mass matrix time taken:  0.004480838775634766
solving Ax = b time taken:  0.14694786071777344
epoch:  207	assembling the mass matrix time taken:  0.0041751861572265625
solving Ax = b time taken:  0.14916753768920898
epoch:  208	assembling the mass matrix time taken:  0.00432276725769043
solving Ax = b time taken:  0.14623332023620605
epoch:  209	assembling the mass matrix time taken:  0.004223823547363281
solving Ax = b time taken:  0.1498725414276123
epoch:  210	assembling the mass matrix time taken:  0.004227876663208008
solving Ax = b time taken:  0.14933490753173828
epoch:  211	assembling the mass matrix time taken:  0.0043103694915771484
solving Ax = b time taken:  0.15113282203674316
epoch:  212	assembling the mass matrix time taken:  0.00477147102355957
solving Ax = b time taken:  0.1477680206298828
epoch:  213	assembling the mass mat

epoch:  273	assembling the mass matrix time taken:  0.004160642623901367
solving Ax = b time taken:  0.2226567268371582
epoch:  274	assembling the mass matrix time taken:  0.00426936149597168
solving Ax = b time taken:  0.2219843864440918
epoch:  275	assembling the mass matrix time taken:  0.004351377487182617
solving Ax = b time taken:  0.22396039962768555
epoch:  276	assembling the mass matrix time taken:  0.004663705825805664
solving Ax = b time taken:  0.21888947486877441
epoch:  277	assembling the mass matrix time taken:  0.004330158233642578
solving Ax = b time taken:  0.2248990535736084
epoch:  278	assembling the mass matrix time taken:  0.004145622253417969
solving Ax = b time taken:  0.22360873222351074
epoch:  279	assembling the mass matrix time taken:  0.004317283630371094
solving Ax = b time taken:  0.22619152069091797
epoch:  280	assembling the mass matrix time taken:  0.0045659542083740234
solving Ax = b time taken:  0.2211897373199463
epoch:  281	assembling the mass matr

epoch:  341	assembling the mass matrix time taken:  0.0041577816009521484
solving Ax = b time taken:  0.27199411392211914
epoch:  342	assembling the mass matrix time taken:  0.0043294429779052734
solving Ax = b time taken:  0.270862340927124
epoch:  343	assembling the mass matrix time taken:  0.004160642623901367
solving Ax = b time taken:  0.2727689743041992
epoch:  344	assembling the mass matrix time taken:  0.004322052001953125
solving Ax = b time taken:  0.26829004287719727
epoch:  345	assembling the mass matrix time taken:  0.004582881927490234
solving Ax = b time taken:  0.27337098121643066
epoch:  346	assembling the mass matrix time taken:  0.005033016204833984
solving Ax = b time taken:  0.27247166633605957
epoch:  347	assembling the mass matrix time taken:  0.00492095947265625
solving Ax = b time taken:  0.27550315856933594
epoch:  348	assembling the mass matrix time taken:  0.004162311553955078
solving Ax = b time taken:  0.27152323722839355
epoch:  349	assembling the mass ma

epoch:  409	assembling the mass matrix time taken:  0.004839897155761719
solving Ax = b time taken:  0.3615703582763672
epoch:  410	assembling the mass matrix time taken:  0.004315853118896484
solving Ax = b time taken:  0.3625960350036621
epoch:  411	assembling the mass matrix time taken:  0.004294872283935547
solving Ax = b time taken:  0.3636443614959717
epoch:  412	assembling the mass matrix time taken:  0.004788637161254883
solving Ax = b time taken:  0.35537147521972656
epoch:  413	assembling the mass matrix time taken:  0.004279136657714844
solving Ax = b time taken:  0.3643829822540283
epoch:  414	assembling the mass matrix time taken:  0.0045893192291259766
solving Ax = b time taken:  0.35933637619018555
epoch:  415	assembling the mass matrix time taken:  0.004262685775756836
solving Ax = b time taken:  0.36549878120422363
epoch:  416	assembling the mass matrix time taken:  0.004133939743041992
solving Ax = b time taken:  0.3526644706726074
epoch:  417	assembling the mass matr

epoch:  477	assembling the mass matrix time taken:  0.004552125930786133
solving Ax = b time taken:  0.41509366035461426
epoch:  478	assembling the mass matrix time taken:  0.004320383071899414
solving Ax = b time taken:  0.41780638694763184
epoch:  479	assembling the mass matrix time taken:  0.004589080810546875
solving Ax = b time taken:  0.4190788269042969
epoch:  480	assembling the mass matrix time taken:  0.004311323165893555
solving Ax = b time taken:  0.4115571975708008
epoch:  481	assembling the mass matrix time taken:  0.004182338714599609
solving Ax = b time taken:  0.4156460762023926
epoch:  482	assembling the mass matrix time taken:  0.004242658615112305
solving Ax = b time taken:  0.4162790775299072
epoch:  483	assembling the mass matrix time taken:  0.004538297653198242
solving Ax = b time taken:  0.42020106315612793
epoch:  484	assembling the mass matrix time taken:  0.004342079162597656
solving Ax = b time taken:  0.413316011428833
epoch:  485	assembling the mass matrix

## $\cos(\pi x_1) \cos( \pi x_2) \cos( \pi x_3) \cos( \pi x_4)$  

In [4]:

def u_exact(x):
    return torch.cos(pi*x[:,0:1])*torch.cos( pi*x[:,1:2]) * torch.cos(pi*x[:,2:3]) * torch.cos(pi*x[:,3:4]) 
def alpha(x): 
#     return torch.ones(x.size(0),1).to(device)
    return 0.5 * torch.sin(6 * pi*x[:,0:1]) + 1. 

def target(x):
#     z = (  4 * (pi)**2 + 1)*torch.cos( pi*x[:,0:1])*torch.cos( pi*x[:,1:2] ) * torch.cos(pi*x[:,2:3]) * torch.cos( pi*x[:,3:4]) 
#     return z 
    z_c = torch.cos( pi*x[:,0:1])*torch.cos( pi*x[:,1:2] ) * torch.cos(pi*x[:,2:3]) * torch.cos( pi*x[:,3:4]) 
    z1 = 3 * pi**2 * torch.sin(pi * x[:,0:1]) * torch.cos( 6*pi*x[:,1:2] ) * torch.cos(pi*x[:,2:3]) * torch.cos( pi*x[:,3:4]) 
    z2 = 0.5 * pi**2 * torch.sin(6*pi * x[:,0:1])* z_c 
    z = z1 + z2 + 3/2*pi**2 * torch.sin(6 * pi * x[:,0:1]) * z_c 
    z += ( 4 * (pi)**2 + 1)*z_c 
    return z 

function_name = "cospix" 
filename_write = "data-neumann/4DOGA-{}-order.txt".format(function_name)
f_write = open(filename_write, "a")
f_write.write("\n")
f_write.close() 
save = False 
for N_list in [[2**2,2**2,2**2,2**2]]: # ,[2**6,2**6],[2**7,2**7] 
    # save = True 
    f_write = open(filename_write, "a")
    my_model = None 
    # Nx = 50   
    # order = 3   
    M = 500000  
    exponent = 8 
    num_epochs = 2**exponent  
    plot_freq = num_epochs 
    N = np.prod(N_list)
    err_QMC2, my_model = OGANeumannReLU4D(my_model,alpha, target,u_exact, N_list,num_epochs,plot_freq, M, k = 1, rand_deter= 'rand', linear_solver = "direct")
    
    if save: 
        folder = 'data-neumann/'
        filename = folder + 'err_OGA_4D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(err_QMC2,filename) 
        folder = 'data-neumann/'
        filename = folder + 'model_OGA_4D_{}_neuron_{}_N_{}_deterministic.pt'.format(function_name,num_epochs,N)
        torch.save(my_model,filename)

    neuron_nums = [2**j for j in range(2,exponent+1)]
    err_list = [err_QMC2[i] for i in neuron_nums ]
    f_write.write('deterministic dictionary size: {}\n'.format(N))
    f_write.write("neuron num \t\t error \t\t order\n")
    print("neuron num \t\t error \t\t order")
    for i, item in enumerate(err_list):
        if i == 0: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t")
            print("*")
            f_write.write("{} \t\t {} \t\t * \n".format(neuron_nums[i],item))
        else: 
            print(neuron_nums[i], end = "\t\t")
            print(item, end = "\t\t") 
            print(np.log(err_list[i-1]/err_list[i])/np.log(2))
            f_write.write("{} \t\t {} \t\t {} \n".format(neuron_nums[i],item,np.log(err_list[i-1]/err_list[i])/np.log(2)))
    f_write.write("\n")
    f_write.close()


using linear solver:  direct
epoch:  1	assembling the mass matrix time taken:  0.0009791851043701172
solving Ax = b time taken:  0.13031697273254395
epoch:  2	assembling the mass matrix time taken:  0.0008194446563720703
solving Ax = b time taken:  0.004035472869873047
epoch:  3	assembling the mass matrix time taken:  0.000885009765625
solving Ax = b time taken:  0.003545045852661133
epoch:  4	assembling the mass matrix time taken:  0.0007338523864746094
solving Ax = b time taken:  0.004254579544067383
epoch:  5	assembling the mass matrix time taken:  0.0009253025054931641
solving Ax = b time taken:  0.0042955875396728516
epoch:  6	assembling the mass matrix time taken:  0.0008869171142578125
solving Ax = b time taken:  0.0046536922454833984
epoch:  7	assembling the mass matrix time taken:  0.0009019374847412109
solving Ax = b time taken:  0.004832267761230469
epoch:  8	assembling the mass matrix time taken:  0.0007388591766357422
solving Ax = b time taken:  0.005568027496337891
epoch:

solving Ax = b time taken:  0.03294014930725098
epoch:  70	assembling the mass matrix time taken:  0.001155853271484375
solving Ax = b time taken:  0.03295540809631348
epoch:  71	assembling the mass matrix time taken:  0.0010988712310791016
solving Ax = b time taken:  0.03379535675048828
epoch:  72	assembling the mass matrix time taken:  0.0010988712310791016
solving Ax = b time taken:  0.03316617012023926
epoch:  73	assembling the mass matrix time taken:  0.0011415481567382812
solving Ax = b time taken:  0.034349679946899414
epoch:  74	assembling the mass matrix time taken:  0.0011341571807861328
solving Ax = b time taken:  0.03452754020690918
epoch:  75	assembling the mass matrix time taken:  0.0013167858123779297
solving Ax = b time taken:  0.03442835807800293
epoch:  76	assembling the mass matrix time taken:  0.0011687278747558594
solving Ax = b time taken:  0.03463125228881836
epoch:  77	assembling the mass matrix time taken:  0.001165628433227539
solving Ax = b time taken:  0.035

epoch:  138	assembling the mass matrix time taken:  0.0011513233184814453
solving Ax = b time taken:  0.07039713859558105
epoch:  139	assembling the mass matrix time taken:  0.001116037368774414
solving Ax = b time taken:  0.07108044624328613
epoch:  140	assembling the mass matrix time taken:  0.0011820793151855469
solving Ax = b time taken:  0.07013225555419922
epoch:  141	assembling the mass matrix time taken:  0.0014891624450683594
solving Ax = b time taken:  0.07105183601379395
epoch:  142	assembling the mass matrix time taken:  0.0012035369873046875
solving Ax = b time taken:  0.0718529224395752
epoch:  143	assembling the mass matrix time taken:  0.13692975044250488
solving Ax = b time taken:  0.050829410552978516
epoch:  144	assembling the mass matrix time taken:  0.0019388198852539062
solving Ax = b time taken:  0.07647323608398438
epoch:  145	assembling the mass matrix time taken:  0.003928184509277344
solving Ax = b time taken:  0.07907772064208984
epoch:  146	assembling the m

epoch:  206	assembling the mass matrix time taken:  0.0013837814331054688
solving Ax = b time taken:  0.1088261604309082
epoch:  207	assembling the mass matrix time taken:  0.0013966560363769531
solving Ax = b time taken:  0.10943388938903809
epoch:  208	assembling the mass matrix time taken:  0.003022909164428711
solving Ax = b time taken:  0.10637426376342773
epoch:  209	assembling the mass matrix time taken:  0.20831036567687988
solving Ax = b time taken:  0.0757606029510498
epoch:  210	assembling the mass matrix time taken:  0.003370523452758789
solving Ax = b time taken:  0.11624550819396973
epoch:  211	assembling the mass matrix time taken:  0.0034568309783935547
solving Ax = b time taken:  0.10921692848205566
epoch:  212	assembling the mass matrix time taken:  0.0015621185302734375
solving Ax = b time taken:  0.10859274864196777
epoch:  213	assembling the mass matrix time taken:  0.0014591217041015625
solving Ax = b time taken:  0.11156225204467773
epoch:  214	assembling the mas

In [5]:
## 